<a href="https://colab.research.google.com/github/thibaudju/velyon/blob/main/python/compteurs_import_retraitements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des fichiers de mesure des compteurs vélo via l'API Grand Lyon

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import requests

X = 1000000
Y = 1
step = 1000000

while True:
    url = f"https://download.data.grandlyon.com/ws/timeseries/pvo_patrimoine_voirie.pvocomptagemeasure/all.csv?maxfeatures={X}&start={Y}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Save the response to a file
        filename = f"/content/drive/MyDrive/Colab Notebooks/mesures_compteurs_velos/response_{Y}_{X+Y-1}.csv"
        with open(filename, 'w') as file:
            file.write(response.text)
        
        # Increase X and Y by the step value
        Y += step
    else:
        break


Export to GCS

In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = 'velyon-batch-1187'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://velyon_batch_1187/


Copy files using gsutil. Use -m tag for multi-threading to increase speed. (There is a Subfolder called "My Drive" that you have to address in your mounted drive)

In [ ]:
!gsutil -m cp -r /content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/* gs://velyon_batch_1187/Comptages/mesures/

Merge into a single CSV

In [ ]:
import pandas as pd

folder_path = '/content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos'

merged_df = pd.DataFrame()


In [ ]:
import os

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        merged_df = merged_df.append(df, ignore_index=True)


In [ ]:
output_file_path = '/content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/merged_mesures.csv'
merged_df.to_csv(output_file_path, index=False)


In [ ]:
#send to GCS

!gsutil -m cp -r /content/drive/MyDrive/Colab_Notebooks/mesures_compteurs_velos/merged_mesures.csv gs://velyon_batch_1187/Comptages/mesures/

# I then imported the csv into a BigQuery table

# Retraitement

Objectif : normalisation des données des compteurs pour effacer le biais lié aux créations / suppressions de nouveaux compteurs et l'installation sur des voies à la fréquentation plus ou moins élevée.

1/ Connexion à BigQuery et import de la table all_mesures

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.cloud import bigquery
client = bigquery.Client(project='velyon-batch-1187')

In [5]:
# Use LIMIT to make your tests if table is very large. Remove it to get the final table
sql_query = ('''SELECT *
                FROM mesures_compteurs.all_mesures
                ''')

df = client.query(sql_query).to_dataframe()

In [ ]:
df

2/ Normalisation des données de mesure 

In [6]:
# Import needed libraries
import numpy as np
from sklearn.preprocessing import MinMaxScaler

We need to normalize the count column, but for each channel_id and not the whole column. 

In [7]:
scaler = MinMaxScaler()

# We group the dataframe by the 'channel_id' column and apply the normalization to the 'count' column within each group

df['normalized_count'] = df.groupby('channel_id')['count'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())

Now let's send that enriched data to BigQuery

First create the new column in the bigquery UI

In [9]:
# convert into csv

df.to_csv("/content/drive/MyDrive/Colab_Notebooks/mesures_normalized.csv")



And send to GCS

In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = 'velyon-batch-1187'
!gcloud config set project {project_id}
!gsutil ls

!gsutil -m cp -r /content/drive/MyDrive/Colab_Notebooks/mesures_normalized.csv gs://velyon_batch_1187/Comptages/mesures/